In [1]:
import numpy as np
# import bayesian_changepoint_detection.online_likelihoods as online_ll
import scipy.special as special

In [2]:
def gamma(z):
    return special.gamma(z)


def t_pdf(x, df, loc=0, scale=1):
    # 如果是 loc=0, scale=1，则退化为标准 t 分布

    # 标准化
    z = (x - loc) / scale

    # 标准 t 分布的 PDF 系数
    numerator = gamma((df + 1) / 2.0)
    denominator = np.sqrt(df * np.pi) * gamma(df / 2.0)
    coefficient = numerator / denominator

    # (1 + z^2/df)^(-(df+1)/2)
    exponent = -(df + 1) / 2.0
    pdf_val_standard = coefficient * (1 + (z**2 / df)) ** exponent

    # loc-scale变换后的 PDF: 对标准 PDF 再除以 scale
    return pdf_val_standard / scale

In [ ]:
class StudentT:
    def __init__(
        self, alpha: float = 0.1, beta: float = 0.1, kappa: float = 1, mu: float = 0
    ):
        """
        StudentT distribution except normal distribution is replaced with the student T distribution
        https://en.wikipedia.org/wiki/Normal-gamma_distribution

        Parameters:
            alpha - alpha in gamma distribution prior
            beta - beta inn gamma distribution prior
            mu - mean from normal distribution
            kappa - variance from normal distribution
        """

        self.alpha0 = self.alpha = np.array([alpha])
        self.beta0 = self.beta = np.array([beta])
        self.kappa0 = self.kappa = np.array([kappa])
        self.mu0 = self.mu = np.array([mu])

    def pdf(self, data: np.array):
        """
        Return the pdf function of the t distribution

        Parmeters:
            data - the datapoints to be evaluated (shape: 1 x D vector)
        """
        return t_pdf(
            x=data,
            df=2 * self.alpha,
            loc=self.mu,
            scale=np.sqrt(self.beta * (self.kappa + 1) / (self.alpha * self.kappa)),
        )

    def update_theta(self, data: np.array, **kwargs):
        """
        Performs a bayesian update on the prior parameters, given data
        Parmeters:
            data - the datapoints to be evaluated (shape: 1 x D vector)
        """
        muT0 = np.concatenate(
            (self.mu0, (self.kappa * self.mu + data) / (self.kappa + 1))
        )
        kappaT0 = np.concatenate((self.kappa0, self.kappa + 1.0))
        alphaT0 = np.concatenate((self.alpha0, self.alpha + 0.5))
        betaT0 = np.concatenate(
            (
                self.beta0,
                self.beta
                + (self.kappa * (data - self.mu) ** 2) / (2.0 * (self.kappa + 1.0)),
            )
        )

        self.mu = muT0
        self.kappa = kappaT0
        self.alpha = alphaT0
        self.beta = betaT0


In [4]:
def online_changepoint_detection(data, hazard_function, log_likelihood_class):
    """
    Use online bayesian changepoint detection
    https://scientya.com/bayesian-online-change-point-detection-an-intuitive-understanding-b2d2b9dc165b

    Parameters:
    data    -- the time series data

    Outputs:
        R  -- is the probability at time step t that the last sequence is already s time steps long
        maxes -- the argmax on column axis of matrix R (growth probability value) for each time step
    """
    maxes = np.zeros(len(data) + 1)

    R = np.zeros((len(data) + 1, len(data) + 1))
    R[0, 0] = 1

    for t, x in enumerate(data):
        # Evaluate the predictive distribution for the new datum under each of
        # the parameters.  This is the standard thing from Bayesian inference.
        predprobs = log_likelihood_class.pdf(x)

        # Evaluate the hazard function for this interval
        H = hazard_function(np.array(range(t + 1)))

        # Evaluate the growth probabilities - shift the probabilities down and to
        # the right, scaled by the hazard function and the predictive
        # probabilities.
        R[1 : t + 2, t + 1] = R[0 : t + 1, t] * predprobs * (1 - H)

        # Evaluate the probability that there *was* a changepoint and we're
        # accumulating the mass back down at r = 0.
        R[0, t + 1] = np.sum(R[0 : t + 1, t] * predprobs * H)

        # Renormalize the run length probabilities for improved numerical
        # stability.
        R[:, t + 1] = R[:, t + 1] / np.sum(R[:, t + 1])

        # Update the parameter sets for each possible run length.
        log_likelihood_class.update_theta(x, t=t)

        maxes[t] = R[:, t].argmax()

    return R, maxes

In [5]:
def constant_hazard(r):
    lam=250
    """
    Hazard function for bayesian online learning
    Arguments:
        lam - inital prob
        r - R matrix
    """
    return 1 / lam * np.ones(r.shape)

def hazard_function(r):
    return constant_hazard(r)
    


In [ ]:
csv_folder = "/home/campus.ncl.ac.uk/c4060464/esp32/microWATCH/datasets/csv/"

# for all files in the folder
import os

for file_name in os.listdir(csv_folder):
    # print("Reading file: ", file_name)
    file_path = csv_folder + file_name
    data = np.loadtxt(file_path, delimiter=",")
    # print("Data shape: ", data.shape)
    # skip if is multi-dimensional
    if len(data.shape) > 1:
        print("Skipping multi-dimensional data: ", data.shape)
        continue

    R, maxes = online_changepoint_detection(
        data, hazard_function, StudentT(alpha=0.1, beta=0.01, kappa=1, mu=0)
    )
    print("R: ", R.shape)
    Nw = 10
    cp_probs = R[Nw, Nw:-1]
    cps = np.where(cp_probs > 0.3)
    print("cps: ", cps)

/tmp/ipykernel_10964/3799741327.py:13: RuntimeWarning: overflow encountered in multiply
  denominator = np.sqrt(df * np.pi) * gamma(df / 2.0)
/tmp/ipykernel_10964/3799741327.py:14: RuntimeWarning: invalid value encountered in divide
  coefficient = numerator / denominator


R:  (582, 582)
cps:  (array([0]),)
R:  (206, 206)
cps:  (array([  0,  12,  59,  74, 140]),)
R:  (101, 101)
cps:  (array([0]),)
R:  (320, 320)
cps:  (array([0]),)
R:  (676, 676)
cps:  (array([0]),)
R:  (16, 16)
cps:  (array([0]),)
R:  (59, 59)
cps:  (array([0]),)
R:  (469, 469)
cps:  (array([0]),)
R:  (331, 331)
cps:  (array([0]),)
R:  (501, 501)
cps:  (array([0]),)
R:  (501, 501)
cps:  (array([  0, 225]),)
R:  (53, 53)
cps:  (array([0]),)
R:  (55, 55)
cps:  (array([0]),)
R:  (482, 482)
cps:  (array([  0,  55,  61,  77, 185, 198, 274]),)
R:  (193, 193)
cps:  (array([0]),)
Skipping multi-dimensional data:  (609, 4)
R:  (284, 284)
cps:  (array([ 0, 97, 98]),)
R:  (302, 302)
cps:  (array([0]),)
Skipping multi-dimensional data:  (622, 2)
Skipping multi-dimensional data:  (376, 2)
R:  (482, 482)
cps:  (array([  0, 106, 251, 265, 266, 280, 292, 307, 325]),)
R:  (215, 215)
cps:  (array([  0, 121, 131, 143, 174]),)
R:  (59, 59)
cps:  (array([0]),)
R:  (216, 216)
cps:  (array([ 0, 80, 99]),)
R: 

In [7]:
from bayesian_changepoint_detection.priors import const_prior # checked
from bayesian_changepoint_detection.offline_likelihoods import IndepentFeaturesLikelihood 
from bayesian_changepoint_detection.bayesian_models import offline_changepoint_detection
from functools import partial

Use scipy logsumexp().


In [8]:

csv_folder = "/home/campus.ncl.ac.uk/c4060464/esp32/microWATCH/datasets/csv/"

# for all files in the folder
import os
for file_name in os.listdir(csv_folder):
    print("Reading file: ", file_name)
    file_path = csv_folder + file_name
    data =np.loadtxt(file_path, delimiter=',')


    Q_ifm, P_ifm, Pcp_ifm = offline_changepoint_detection(
        data, partial(const_prior, p=1/(len(data) + 1)), IndepentFeaturesLikelihood(), truncate=-20
    )   

Reading file:  bank.csv


/home/campus.ncl.ac.uk/c4060464/esp32/microWATCH/src/algorithms/bayesian_changepoint_detection/bayesian_changepoint_detection/offline_likelihoods.py:73: RuntimeWarning: divide by zero encountered in log
  + (N0 / 2) * np.log(V0)
/home/campus.ncl.ac.uk/c4060464/esp32/microWATCH/src/algorithms/bayesian_changepoint_detection/bayesian_changepoint_detection/bayesian_models.py:57: RuntimeWarning: invalid value encountered in scalar subtract
  if summand - P_next_cp < truncate:


Reading file:  shanghai_license.csv
Reading file:  nile.csv
Reading file:  construction.csv
Reading file:  well_log.csv
Reading file:  centralia.csv
Reading file:  gdp_japan.csv
Reading file:  jfk_passengers.csv
Reading file:  businv.csv
Reading file:  quality_control_4.csv
Reading file:  brent_spot.csv
Reading file:  robocalls.csv
Reading file:  ozone.csv
Reading file:  scanline_42049.csv
Reading file:  seatbelts.csv
Reading file:  bee_waggle_6.csv
Reading file:  quality_control_2.csv
Reading file:  children_per_woman.csv


/home/campus.ncl.ac.uk/c4060464/esp32/microWATCH/src/algorithms/bayesian_changepoint_detection/bayesian_changepoint_detection/bayesian_models.py:79: RuntimeWarning: invalid value encountered in subtract
  Pcp[j - 1, j - 1 : t]


Reading file:  apple.csv
Reading file:  run_log.csv
Reading file:  scanline_126007.csv
Reading file:  unemployment_nl.csv
Reading file:  gdp_iran.csv
Reading file:  co2_canada.csv
Reading file:  lga_passengers.csv
Reading file:  rail_lines.csv
Reading file:  occupancy.csv
Reading file:  uk_coal_employ.csv
Reading file:  bitcoin.csv


/home/campus.ncl.ac.uk/c4060464/esp32/microWATCH/src/algorithms/bayesian_changepoint_detection/bayesian_changepoint_detection/bayesian_models.py:53: RuntimeWarning: invalid value encountered in logaddexp
  P_next_cp = np.logaddexp(P_next_cp, summand)
/home/campus.ncl.ac.uk/c4060464/esp32/microWATCH/src/algorithms/bayesian_changepoint_detection/bayesian_changepoint_detection/bayesian_models.py:69: RuntimeWarning: invalid value encountered in logaddexp
  Q[t] = np.logaddexp(P_next_cp, P[t, n - 1] + antiG)


Reading file:  quality_control_3.csv
Reading file:  measles.csv
Reading file:  us_population.csv
Reading file:  gdp_argentina.csv
Reading file:  quality_control_5.csv
Reading file:  debt_ireland.csv
Reading file:  gdp_croatia.csv
Reading file:  ratner_stock.csv
Reading file:  quality_control_1.csv
Reading file:  usd_isk.csv
Reading file:  iceland_tourism.csv
Reading file:  homeruns.csv
Reading file:  global_co2.csv
